# IVI Data Exploration and Cleaning Pipeline

This notebook covers:
1. Loading raw SAS datasets efficiently with Polars + pyreadstat
2. Initial exploration (shape, dtypes, samples)
3. Missing value and outlier analysis
4. **Multi-level data curation:**
   - Contract-level aggregates (for IVI scoring)
   - Member-level features (for detailed analysis)
   - Dimension tables (nationality, provider, diagnosis - for drill-downs)
5. Saving optimized parquet files for downstream analysis

**Design Principles:**
- Use Polars for speed and memory efficiency
- Preserve granularity through multiple aggregation levels
- Create analysis-ready tables for visualizations

In [4]:
# Import Required Libraries
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import gc
import multiprocessing

# Fast SAS reading
try:
    import pyreadstat
    USE_PYREADSTAT = True
    print('pyreadstat available - using fast SAS reader')
except ImportError:
    USE_PYREADSTAT = False
    print('pyreadstat not available - install with: pip install pyreadstat')

warnings.filterwarnings('ignore')

# Polars config for better display
pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(20)
pl.Config.set_fmt_str_lengths(50)

# Define paths
DATA_DIR = Path('/volume/data/KAU-Bupa')
OUTPUT_DIR = Path('/volume/data/processed')
CACHE_DIR = Path('/volume/data/cache')

for d in [OUTPUT_DIR, CACHE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

NUM_CORES = multiprocessing.cpu_count()

print(f'Polars version: {pl.__version__}')
print(f'CPU cores available: {NUM_CORES}')
print(f'Data directory: {DATA_DIR}')
print(f'Output directory: {OUTPUT_DIR}')

pyreadstat available - using fast SAS reader
Polars version: 1.37.1
CPU cores available: 46
Data directory: /volume/data/KAU-Bupa
Output directory: /volume/data/processed


## 1. Data Loading Strategy

**Approach:**
1. Use `pyreadstat.read_file_multiprocessing` for fast SAS reading
2. Convert to Polars LazyFrame immediately
3. Cache as Parquet for instant future loads
4. Use lazy evaluation where possible to minimize memory

In [ ]:
def load_sas_to_polars(sas_path: Path, cache_path: Path, name: str, use_cache: bool = True) -> pl.LazyFrame:
    """
    Load SAS file and return as Polars LazyFrame.
    Uses pyreadstat multiprocessing + parquet caching.
    """
    cache_file = cache_path / f'{name}.parquet'
    
    # Check cache first
    if use_cache and cache_file.exists():
        print(f'[{name}] Loading from parquet cache...')
        return pl.scan_parquet(cache_file)
    
    # Read from SAS
    print(f'[{name}] Reading SAS file with {NUM_CORES} cores...')
    
    if USE_PYREADSTAT:
        df_pd, meta = pyreadstat.read_file_multiprocessing(
            pyreadstat.read_sas7bdat,
            str(sas_path),
            num_processes=NUM_CORES
        )
        # Convert to Polars
        df = pl.from_pandas(df_pd)
        del df_pd
        gc.collect()
    else:
        # Fallback: pandas then polars
        df_pd = pd.read_sas(sas_path, encoding='latin1')
        df = pl.from_pandas(df_pd)
        del df_pd
        gc.collect()
    
    print('done reading')
    # Standardize column names
    df = df.rename({col: col.strip().upper() for col in df.columns})
    
    # Decode bytes columns
    for col in df.columns:
        if df[col].dtype == pl.Object:
            df = df.with_columns(
                pl.col(col).map_elements(
                    lambda x: x.decode('latin1') if isinstance(x, bytes) else x,
                    return_dtype=pl.Utf8
                ).alias(col)
            )
    
    print(f'[{name}] Shape: {df.shape}')
    
    # Cache to parquet
    if use_cache:
        print(f'[{name}] Caching to parquet...')
        df.write_parquet(cache_file)
        print(f'[{name}] Cached to: {cache_file}')
        # Return as lazy for memory efficiency
        return pl.scan_parquet(cache_file)
    
    return df.lazy()


print('Loading functions defined.')

Loading functions defined.


In [11]:
# Load all datasets as Polars LazyFrames
print('='*60)
print('LOADING DATASETS')
print('='*60)

lf_member = load_sas_to_polars(DATA_DIR / 'sampled_member.sas7bdat', CACHE_DIR, 'member')
lf_calls = load_sas_to_polars(DATA_DIR / 'sampled_calls.sas7bdat', CACHE_DIR, 'calls')
lf_claims = load_sas_to_polars(DATA_DIR / 'sampled_claims.sas7bdat', CACHE_DIR, 'claims')
lf_preauth = load_sas_to_polars(DATA_DIR / 'sampled_preauth.sas7bdat', CACHE_DIR, 'preauth')

print('\nAll datasets loaded as LazyFrames (memory efficient).')

LOADING DATASETS
[member] Loading from parquet cache...
[calls] Loading from parquet cache...
[claims] Loading from parquet cache...
[preauth] Loading from parquet cache...

All datasets loaded as LazyFrames (memory efficient).


In [12]:
# Load provider reference data (small Excel file)
df_provider = pl.read_excel(DATA_DIR / 'Provider_Info.xlsx')
df_provider = df_provider.rename({col: col.strip().upper() for col in df_provider.columns})

print(f'Provider Info: {df_provider.shape}')
print(f'Columns: {df_provider.columns}')
df_provider.head()

Provider Info: (3558, 6)
Columns: ['PROV_CODE', 'PROV_NAME', 'PROVIDER_NETWORK', 'PROVIDER_PRACTICE', 'PROVIDER_REGION', 'PROVIDER_TOWN']


PROV_CODE,PROV_NAME,PROVIDER_NETWORK,PROVIDER_PRACTICE,PROVIDER_REGION,PROVIDER_TOWN
str,str,str,str,str,str
"""20068""","""Dallah Hospital - Nakeel""","""H. NW7""","""Hospital""","""Central""","""Riyadh"""
"""20134""","""Saudi German Hospitals Group""","""G. NW6""","""Hospital""","""Western""","""Jeddah"""
"""20147""","""United Doctors Hospital""","""F. NW5""","""Hospital""","""Western""","""Jeddah"""
"""22478""","""Oral Dental Clinics""","""J. ONW""","""Dental""","""Western""","""Jeddah"""
"""22671""","""Al Nahdi Medical Company""","""F. NW5""","""Pharmacy""","""Western""","""Jeddah"""


In [13]:
# Quick schema inspection (no data loading - just metadata)
print('DATASET SCHEMAS')
print('='*60)

for name, lf in [('Members', lf_member), ('Calls', lf_calls), 
                  ('Claims', lf_claims), ('Preauth', lf_preauth)]:
    print(f'\n{name}:')
    print(f'  Columns: {lf.collect_schema().names()}')
    print(f'  Types: {dict(lf.collect_schema())}')

DATASET SCHEMAS

Members:
  Columns: ['ADHERENT_NO', 'CONT_YYMM', 'PLAN_ID', 'CONTRACT_NO', 'PLAN_NETWORK', 'GENDER', 'NATIONALITY', 'WE', 'WP']
  Types: {'ADHERENT_NO': String, 'CONT_YYMM': String, 'PLAN_ID': String, 'CONTRACT_NO': String, 'PLAN_NETWORK': String, 'GENDER': String, 'NATIONALITY': String, 'WE': Float64, 'WP': Float64}

Calls:
  Columns: ['CONT_NO', 'CONT_YYMM', 'MBR_NO', 'CALL_ID', 'STATUS', 'CALL_CAT', 'CRT_DATE', 'UPD_DATE']
  Types: {'CONT_NO': String, 'CONT_YYMM': String, 'MBR_NO': String, 'CALL_ID': Float64, 'STATUS': String, 'CALL_CAT': String, 'CRT_DATE': Datetime(time_unit='us', time_zone=None), 'UPD_DATE': Datetime(time_unit='us', time_zone=None)}

Claims:
  Columns: ['ADHERENT_NO', 'PLAN_ID', 'CONT_NO', 'CONT_YYMM', 'VOU_NO', 'LINE_NO', 'PROV_CODE', 'T_PERIOD', 'INCUR_DATE_FROM', 'PROCESS_DATE', 'CLAIM_TYPE', 'DIAG_CODE', 'BEN_HEAD', 'SUBMIT_BY', 'STATUS', 'SUM_OF_NETBILLED']
  Types: {'ADHERENT_NO': String, 'PLAN_ID': String, 'CONT_NO': String, 'CONT_YYMM': S

## 2. Initial Data Exploration

Using Polars lazy evaluation to get statistics without loading full data into memory.

In [14]:
def explore_lazy(lf: pl.LazyFrame, name: str):
    """Explore a LazyFrame with minimal memory usage."""
    print('='*60)
    print(f'Dataset: {name}')
    print('='*60)
    
    # Get row count
    row_count = lf.select(pl.len()).collect().item()
    print(f'Rows: {row_count:,}')
    print(f'Columns: {len(lf.collect_schema())}')
    
    # Sample data
    print('\nSample (5 rows):')
    display(lf.head(5).collect())
    
    # Null counts
    print('\nNull counts:')
    null_counts = lf.null_count().collect()
    display(null_counts)
    
    return row_count

In [15]:
# Explore each dataset
n_members = explore_lazy(lf_member, 'Members')

Dataset: Members
Rows: 4,263,550
Columns: 9

Sample (5 rows):


ADHERENT_NO,CONT_YYMM,PLAN_ID,CONTRACT_NO,PLAN_NETWORK,GENDER,NATIONALITY,WE,WP
str,str,str,str,str,str,str,f64,f64
"""0110058600_0100011213_202201""","""202201""","""0222320""","""0110058600""","""G. NW6""","""F""","""Saudi Arabia""",1.0,20688.822
"""0110058600_0100011312_202201""","""202201""","""0222320""","""0110058600""","""G. NW6""","""F""","""Saudi Arabia""",1.0,20688.822
"""0110058600_0100011320_202201""","""202201""","""0222320""","""0110058600""","""G. NW6""","""M""","""Saudi Arabia""",1.0,20688.822
"""0110058600_0100022210_202201""","""202201""","""0222322""","""0110058600""","""C. NW2""","""F""","""Pakistan""",1.0,10444.357
"""0110058600_0100022319_202201""","""202201""","""0222322""","""0110058600""","""C. NW2""","""F""","""Pakistan""",1.0,10444.357



Null counts:


ADHERENT_NO,CONT_YYMM,PLAN_ID,CONTRACT_NO,PLAN_NETWORK,GENDER,NATIONALITY,WE,WP
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0


In [16]:
n_claims = explore_lazy(lf_claims, 'Claims')

Dataset: Claims
Rows: 83,288,467
Columns: 16

Sample (5 rows):


ADHERENT_NO,PLAN_ID,CONT_NO,CONT_YYMM,VOU_NO,LINE_NO,PROV_CODE,T_PERIOD,INCUR_DATE_FROM,PROCESS_DATE,CLAIM_TYPE,DIAG_CODE,BEN_HEAD,SUBMIT_BY,STATUS,SUM_OF_NETBILLED
str,str,str,str,str,f64,str,str,datetime[μs],datetime[μs],str,str,str,str,str,f64
"""0110013300_0100016001_202305""","""0237733""","""0110013300""","""202305""","""01232981625""",1.0,"""22075""","""202306""",2023-06-16 00:00:00,2023-07-05 00:00:00,"""O""","""M79.71""","""Standard Chronic""","""P""","""AC""",36.344
"""0110013300_0100016001_202305""","""0237733""","""0110013300""","""202305""","""01232981625""",2.0,"""22075""","""202306""",2023-06-16 00:00:00,2023-07-05 00:00:00,"""O""","""M79.71""","""Standard Chronic""","""P""","""AC""",6.85993
"""0110013300_0100016001_202305""","""0237733""","""0110013300""","""202305""","""01232981625""",3.0,"""22075""","""202306""",2023-06-16 00:00:00,2023-07-05 00:00:00,"""O""","""M79.71""","""Standard Chronic""","""P""","""AC""",43.6128
"""0110013300_0100016001_202305""","""0237733""","""0110013300""","""202305""","""01232981625""",4.0,"""22075""","""202306""",2023-06-16 00:00:00,2023-07-05 00:00:00,"""O""","""M79.71""","""Standard Chronic""","""P""","""AC""",229.01263
"""0110013300_0100016001_202305""","""0237733""","""0110013300""","""202305""","""01232981625""",5.0,"""22075""","""202306""",2023-06-16 00:00:00,2023-07-05 00:00:00,"""O""","""M79.71""","""Standard Chronic""","""P""","""AC""",77.231



Null counts:


ADHERENT_NO,PLAN_ID,CONT_NO,CONT_YYMM,VOU_NO,LINE_NO,PROV_CODE,T_PERIOD,INCUR_DATE_FROM,PROCESS_DATE,CLAIM_TYPE,DIAG_CODE,BEN_HEAD,SUBMIT_BY,STATUS,SUM_OF_NETBILLED
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
n_calls = explore_lazy(lf_calls, 'Calls')

Dataset: Calls
Rows: 8,920,852
Columns: 8

Sample (5 rows):


CONT_NO,CONT_YYMM,MBR_NO,CALL_ID,STATUS,CALL_CAT,CRT_DATE,UPD_DATE
str,str,str,f64,str,str,datetime[μs],datetime[μs]
"""0144772700""","""202207""","""0100220017""",1.4598664e7,"""C""","""BR""",2018-06-03 10:17:16,2022-07-26 08:40:05
"""0141687800""","""202103""","""01""",1.3728905e7,"""C""","""BO""",2018-01-15 09:52:22,2021-04-26 11:16:40
"""0142979100""","""202107""","""0100067029""",1.4520552e7,"""C""","""RE""",2018-05-20 13:41:58,2021-08-17 09:59:10
"""0144740100""","""202206""","""0100827020""",1.4760342e7,"""C""","""BR""",2018-07-04 15:35:49,2022-07-26 08:39:50
"""0129784600""","""202201""","""0100200014""",1.4356306e7,"""C""","""VP""",2018-04-23 16:40:33,2022-05-18 17:30:44



Null counts:


CONT_NO,CONT_YYMM,MBR_NO,CALL_ID,STATUS,CALL_CAT,CRT_DATE,UPD_DATE
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0


In [18]:
n_preauth = explore_lazy(lf_preauth, 'Preauth')

Dataset: Preauth
Rows: 38,188,288
Columns: 14

Sample (5 rows):


CONT_NO,CONT_YYMM,MBR_NO,PLAN_ID,PREAUTH_EPISODE_ID,PREAUTH_EPISODE_ITEM_ID,TREATMENT_DATE,PROV_CODE,EPISODE_STATUS (STATUS),LINE_STATUS (STATUS),ICD_CODE (DIAG_CODE),BEN_HEAD,REJ_REASON_ID,EST_AMT
str,str,str,str,f64,f64,datetime[μs],str,str,str,str,str,f64,f64
"""0110013300""","""202205""","""0100016327""","""0226865""",7.7879241e7,1.96624054e8,2022-06-21 00:00:00,"""22825""","""C""","""A""","""M54.2""","""C1""",null,331.639
"""0110013300""","""202205""","""0100016327""","""0226865""",7.7879241e7,1.96624055e8,2022-06-21 00:00:00,"""22825""","""C""","""R""","""M79.2""","""C1""",19.0,115.8465
"""0110013300""","""202205""","""0100016327""","""0226865""",7.7879241e7,1.96624056e8,2022-06-21 00:00:00,"""22825""","""C""","""A""","""M54.2""","""C1""",null,104.489
"""0110013300""","""202205""","""0100016327""","""0226865""",7.7879241e7,1.96624057e8,2022-06-21 00:00:00,"""22825""","""C""","""A""","""R51""","""S1""",null,104.489
"""0110013300""","""202205""","""0100016327""","""0226865""",7.793828e7,1.96769569e8,2022-06-22 00:00:00,"""22825""","""C""","""A""","""M79.2""","""C1""",null,115.8465



Null counts:


CONT_NO,CONT_YYMM,MBR_NO,PLAN_ID,PREAUTH_EPISODE_ID,PREAUTH_EPISODE_ITEM_ID,TREATMENT_DATE,PROV_CODE,EPISODE_STATUS (STATUS),LINE_STATUS (STATUS),ICD_CODE (DIAG_CODE),BEN_HEAD,REJ_REASON_ID,EST_AMT
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,27595621,1483


## 3. Key Statistics (Lazy Computation)

In [19]:
# Key statistics computed lazily
print('KEY STATISTICS')
print('='*60)

# Members stats
member_stats = lf_member.select([
    pl.col('ADHERENT_NO').n_unique().alias('unique_members'),
    pl.col('CONTRACT_NO').n_unique().alias('unique_contracts'),
    pl.col('PLAN_ID').n_unique().alias('unique_plans'),
    pl.col('WP').sum().alias('total_premium'),
    pl.col('WE').sum().alias('total_earned'),
]).collect()

print('\nMEMBERS:')
print(f"  Unique members: {member_stats['unique_members'].item():,}")
print(f"  Unique contracts: {member_stats['unique_contracts'].item():,}")
print(f"  Unique plans: {member_stats['unique_plans'].item():,}")
print(f"  Total premium: {member_stats['total_premium'].item():,.0f}")

# Claims stats
claims_stats = lf_claims.select([
    pl.col('ADHERENT_NO').n_unique().alias('unique_members'),
    pl.col('CONT_NO').n_unique().alias('unique_contracts'),
    pl.col('VOU_NO').n_unique().alias('unique_vouchers'),
    pl.col('PROV_CODE').n_unique().alias('unique_providers'),
    pl.col('SUM_OF_NETBILLED').sum().alias('total_billed'),
]).collect()

print('\nCLAIMS:')
print(f"  Unique members: {claims_stats['unique_members'].item():,}")
print(f"  Unique contracts: {claims_stats['unique_contracts'].item():,}")
print(f"  Unique vouchers: {claims_stats['unique_vouchers'].item():,}")
print(f"  Total billed: {claims_stats['total_billed'].item():,.0f}")

KEY STATISTICS

MEMBERS:
  Unique members: 4,201,467
  Unique contracts: 133,704
  Unique plans: 17,074
  Total premium: 69,386,332,785

CLAIMS:
  Unique members: 2,523,830
  Unique contracts: 32,987
  Unique vouchers: 18,386,312
  Total billed: 53,712,680,255


In [20]:
# Calls and Preauth stats
calls_stats = lf_calls.select([
    pl.col('CONT_NO').n_unique().alias('unique_contracts'),
    pl.col('CALL_ID').n_unique().alias('unique_calls'),
    pl.col('MBR_NO').n_unique().alias('unique_members'),
]).collect()

print('CALLS:')
print(f"  Unique contracts: {calls_stats['unique_contracts'].item():,}")
print(f"  Unique calls: {calls_stats['unique_calls'].item():,}")

preauth_stats = lf_preauth.select([
    pl.col('CONT_NO').n_unique().alias('unique_contracts'),
    pl.col('PREAUTH_EPISODE_ID').n_unique().alias('unique_episodes'),
    pl.col('MBR_NO').n_unique().alias('unique_members'),
    pl.col('EST_AMT').sum().alias('total_estimated'),
]).collect()

print('\nPREAUTH:')
print(f"  Unique contracts: {preauth_stats['unique_contracts'].item():,}")
print(f"  Unique episodes: {preauth_stats['unique_episodes'].item():,}")
print(f"  Total estimated: {preauth_stats['total_estimated'].item():,.0f}")

CALLS:
  Unique contracts: 46,570
  Unique calls: 8,920,852

PREAUTH:
  Unique contracts: 32,573
  Unique episodes: 15,593,968
  Total estimated: 116,400,633,936


## 4. Multi-Level Data Aggregation Strategy

We create **three levels** of aggregated data to preserve information:

1. **Contract-Level** - For IVI scoring and corporate client analysis
2. **Member-Level** - For individual health patterns and demographics
3. **Dimension Tables** - For drill-down analysis by nationality, provider, diagnosis, etc.

### 4.1 Contract-Level Aggregation (for IVI)

In [34]:
# Contract-level aggregation from Members
print('Aggregating Members at Contract level...')

agg_member_contract = lf_member.group_by('CONTRACT_NO').agg([
    pl.col('ADHERENT_NO').n_unique().alias('TOTAL_MEMBERS'),
    pl.col('PLAN_ID').n_unique().alias('TOTAL_PLANS'),
    pl.col('WP').sum().alias('TOTAL_WRITTEN_PREMIUM'),
    pl.col('WE').sum().alias('TOTAL_EARNED'),
    pl.col('PLAN_NETWORK').n_unique().alias('UNIQUE_NETWORKS'),
    pl.col('NATIONALITY').n_unique().alias('UNIQUE_NATIONALITIES'),
    # Gender counts
    (pl.col('GENDER') == 'M').sum().alias('MALE_COUNT'),
    (pl.col('GENDER') == 'F').sum().alias('FEMALE_COUNT'),
    # Contract time span
    pl.col('CONT_YYMM').min().alias('CONTRACT_START'),
    pl.col('CONT_YYMM').max().alias('CONTRACT_END'),
]).collect()

print(f'Contract-Member aggregation: {agg_member_contract.shape}')
agg_member_contract.sort(pl.col("UNIQUE_NATIONALITIES"), descending=True).head(15)

Aggregating Members at Contract level...


Contract-Member aggregation: (133704, 11)


CONTRACT_NO,TOTAL_MEMBERS,TOTAL_PLANS,TOTAL_WRITTEN_PREMIUM,TOTAL_EARNED,UNIQUE_NETWORKS,UNIQUE_NATIONALITIES,MALE_COUNT,FEMALE_COUNT,CONTRACT_START,CONTRACT_END
str,u32,u32,f64,f64,u32,u32,u32,u32,str,str
"""0110525300""",11432,13,3.4434e8,9614.11,1,119,5952,5542,"""202211""","""202311"""
"""0132559700""",126485,27,8.4357e9,120805.66,2,95,63369,63133,"""202207""","""202307"""
"""0145914700""",11265,6,1.9405e8,9623.42,2,85,6853,4451,"""202201""","""202301"""
"""0150564600""",3995,3,5.5981e7,2943.54,3,84,3038,1048,"""202308""","""202308"""
"""0145218500""",5419,6,1.1622e8,3886.69,3,84,3839,1621,"""202201""","""202301"""
"""0144579200""",7391,25,3.0026e8,5220.9,2,82,3896,3511,"""202206""","""202306"""
"""0129214100""",21706,10,3.6332e8,19715.49,3,80,12601,9324,"""202201""","""202301"""
"""0149989700""",6059,10,1.0084e8,5075.62,3,77,3422,2815,"""202211""","""202311"""
"""0145854900""",7406,20,9.6397e7,5022.6,6,75,5077,2763,"""202211""","""202311"""


In [35]:
# Contract-level aggregation from Claims
print('Aggregating Claims at Contract level...')

agg_claims_contract = lf_claims.group_by('CONT_NO').agg([
    pl.len().alias('TOTAL_CLAIM_LINES'),
    pl.col('VOU_NO').n_unique().alias('UNIQUE_CLAIMS'),
    pl.col('SUM_OF_NETBILLED').sum().alias('TOTAL_NET_BILLED'),
    pl.col('SUM_OF_NETBILLED').mean().alias('AVG_NET_BILLED'),
    pl.col('SUM_OF_NETBILLED').max().alias('MAX_NET_BILLED'),
    pl.col('SUM_OF_NETBILLED').std().alias('STD_NET_BILLED'),
    pl.col('ADHERENT_NO').n_unique().alias('MEMBERS_WITH_CLAIMS'),
    pl.col('PROV_CODE').n_unique().alias('UNIQUE_PROVIDERS'),
    pl.col('DIAG_CODE').n_unique().alias('UNIQUE_DIAGNOSES'),
    pl.col('BEN_HEAD').n_unique().alias('UNIQUE_BENEFIT_HEADS'),
    # Time features
    pl.col('INCUR_DATE_FROM').min().alias('FIRST_CLAIM_DATE'),
    pl.col('INCUR_DATE_FROM').max().alias('LAST_CLAIM_DATE'),
]).rename({'CONT_NO': 'CONTRACT_NO'}).collect()

print(f'Contract-Claims aggregation: {agg_claims_contract.shape}')
agg_claims_contract.head()

Aggregating Claims at Contract level...


Contract-Claims aggregation: (32987, 13)


CONTRACT_NO,TOTAL_CLAIM_LINES,UNIQUE_CLAIMS,TOTAL_NET_BILLED,AVG_NET_BILLED,MAX_NET_BILLED,STD_NET_BILLED,MEMBERS_WITH_CLAIMS,UNIQUE_PROVIDERS,UNIQUE_DIAGNOSES,UNIQUE_BENEFIT_HEADS,FIRST_CLAIM_DATE,LAST_CLAIM_DATE
str,u32,u32,f64,f64,f64,f64,u32,u32,u32,u32,datetime[μs],datetime[μs]
"""0156021000""",116,20,26631.56573,229.582463,1904.15302,286.963895,2,7,19,2,2023-10-11 00:00:00,2024-09-11 00:00:00
"""0150065300""",144,29,11072.83562,76.894692,461.20536,65.821307,3,2,20,2,2023-01-06 00:00:00,2023-11-11 00:00:00
"""0155090600""",1271,269,386098.93938,303.775719,50570.35907,1991.088493,64,63,134,21,2023-01-18 00:00:00,2024-08-21 00:00:00
"""0149716900""",33,9,27063.65046,820.11062,3422.37819,939.348697,2,3,7,4,2023-01-16 00:00:00,2023-07-09 00:00:00
"""0155174500""",1541,287,344286.39399,223.417517,33786.291,1014.114661,19,18,127,17,2023-01-02 00:00:00,2023-11-27 00:00:00


In [36]:
# Contract-level aggregation from Calls
print('Aggregating Calls at Contract level...')

agg_calls_contract = lf_calls.group_by('CONT_NO').agg([
    pl.len().alias('TOTAL_CALLS'),
    pl.col('CALL_ID').n_unique().alias('UNIQUE_CALLS'),
    pl.col('CALL_CAT').n_unique().alias('UNIQUE_CALL_CATEGORIES'),
    pl.col('MBR_NO').n_unique().alias('UNIQUE_CALLERS'),
]).rename({'CONT_NO': 'CONTRACT_NO'}).collect()

print(f'Contract-Calls aggregation: {agg_calls_contract.shape}')
agg_calls_contract.head()

Aggregating Calls at Contract level...
Contract-Calls aggregation: (46570, 5)


CONTRACT_NO,TOTAL_CALLS,UNIQUE_CALLS,UNIQUE_CALL_CATEGORIES,UNIQUE_CALLERS
str,u32,u32,u32,u32
"""0148760500""",32,32,5,7
"""0147533700""",3,3,2,3
"""0149800200""",82,82,5,10
"""0148308600""",67,67,6,6
"""0149336600""",12,12,1,3


In [37]:
# Contract-level aggregation from Preauth
print('Aggregating Preauth at Contract level...')

agg_preauth_contract = lf_preauth.group_by('CONT_NO').agg([
    pl.len().alias('TOTAL_PREAUTH_ITEMS'),
    pl.col('PREAUTH_EPISODE_ID').n_unique().alias('UNIQUE_PREAUTH_EPISODES'),
    pl.col('MBR_NO').n_unique().alias('MEMBERS_WITH_PREAUTH'),
    pl.col('EST_AMT').sum().alias('TOTAL_ESTIMATED_AMT'),
    pl.col('EST_AMT').mean().alias('AVG_ESTIMATED_AMT'),
    pl.col('PROV_CODE').n_unique().alias('UNIQUE_PROVIDERS_PREAUTH'),
]).rename({'CONT_NO': 'CONTRACT_NO'}).collect()

print(f'Contract-Preauth aggregation: {agg_preauth_contract.shape}')
agg_preauth_contract.head()

Aggregating Preauth at Contract level...
Contract-Preauth aggregation: (32573, 7)


CONTRACT_NO,TOTAL_PREAUTH_ITEMS,UNIQUE_PREAUTH_EPISODES,MEMBERS_WITH_PREAUTH,TOTAL_ESTIMATED_AMT,AVG_ESTIMATED_AMT,UNIQUE_PROVIDERS_PREAUTH
str,u32,u32,u32,f64,f64,u32
"""0150658000""",43,20,3,23596.25114,548.750027,5
"""0155705200""",51,22,3,24090.6204,472.365106,5
"""0155744900""",74,28,8,38121.90305,515.160852,13
"""0144176100""",110,38,3,43939.80514,399.452774,5
"""0149156000""",71,30,4,9140.516,128.739662,2


In [39]:
# Preauth status rates per contract
print('Computing preauth status rates...')

preauth_status_rates = lf_preauth.group_by(['CONT_NO', 'EPISODE_STATUS (STATUS)']).agg([
    pl.len().alias('count')
]).collect().pivot(
    on='EPISODE_STATUS (STATUS)',
    index='CONT_NO',
    values='count'
).fill_null(0)

# Add total and compute rates
status_cols = [c for c in preauth_status_rates.columns if c != 'CONT_NO']
preauth_status_rates = preauth_status_rates.with_columns([
    pl.sum_horizontal(status_cols).alias('TOTAL_STATUS')
])

for col in status_cols:
    preauth_status_rates = preauth_status_rates.with_columns([
        (pl.col(col) / pl.col('TOTAL_STATUS')).alias(f'{col}_RATE')
    ])

preauth_status_rates = preauth_status_rates.rename({'CONT_NO': 'CONTRACT_NO'})
print(f'Preauth status rates: {preauth_status_rates.shape}')
preauth_status_rates.head()

Computing preauth status rates...


Preauth status rates: (32573, 26)


CONTRACT_NO,C,N,P,D,R,A,K,U,T,…,P_RATE,D_RATE,R_RATE,A_RATE,K_RATE,U_RATE,T_RATE,H_RATE,_RATE,W_RATE
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""0149732900""",121,0,24,4,123,5,41,0,0,…,0.075472,0.012579,0.386792,0.015723,0.128931,0.0,0.0,0.0,0.0,0.0
"""0150390300""",3210,78,42,38,893,218,560,182,0,…,0.008044,0.007278,0.17104,0.041754,0.107259,0.034859,0.0,0.0,0.0,0.0
"""0144691400""",1211,1,56,0,870,131,72,148,0,…,0.022499,0.0,0.349538,0.052632,0.028927,0.059462,0.0,0.0,0.0,0.0
"""0148978900""",47,0,0,0,38,0,0,7,0,…,0.0,0.0,0.413043,0.0,0.0,0.076087,0.0,0.0,0.0,0.0
"""0149505000""",2957,0,252,43,1224,376,225,251,0,…,0.047288,0.008069,0.229687,0.070557,0.042222,0.047101,0.0,0.0,0.0,0.000188


In [ ]:
### 4.2 Member-Level Aggregation (preserves individual patterns)

In [40]:
# Member-level base info (demographics)
print('Creating member-level dataset...')

member_base = lf_member.group_by('ADHERENT_NO').agg([
    pl.col('CONTRACT_NO').first().alias('CONTRACT_NO'),
    pl.col('PLAN_ID').first().alias('PLAN_ID'),
    pl.col('GENDER').first().alias('GENDER'),
    pl.col('NATIONALITY').first().alias('NATIONALITY'),
    pl.col('PLAN_NETWORK').first().alias('PLAN_NETWORK'),
    pl.col('WP').sum().alias('TOTAL_PREMIUM'),
    pl.col('WE').sum().alias('TOTAL_EARNED'),
    pl.col('CONT_YYMM').n_unique().alias('MONTHS_ENROLLED'),
]).collect()

print(f'Member base: {member_base.shape}')
member_base.head()

Creating member-level dataset...
Member base: (4201467, 9)


ADHERENT_NO,CONTRACT_NO,PLAN_ID,GENDER,NATIONALITY,PLAN_NETWORK,TOTAL_PREMIUM,TOTAL_EARNED,MONTHS_ENROLLED
str,str,str,str,str,str,f64,f64,u32
"""0145432200_0102868081_202204""","""0145432200""","""0225834""","""M""","""Saudi Arabia""","""F. NW5""",11861.773,1.0,1
"""0149108100_0101984001_202303""","""0149108100""","""0236040""","""M""","""Saudi Arabia""","""G. NW6""",2970.2134,0.22,1
"""0148025900_0102210021_202305""","""0148025900""","""0238182""","""F""","""Saudi Arabia""","""B. NW1""",17913.049,1.0,1
"""0147789600_0105730012_202202""","""0147789600""","""0224732""","""M""","""Nepal""","""NWM""",3352.734,1.0,1
"""0139154000_0100272013_202208""","""0139154000""","""0229833""","""M""","""Egypt""","""A. NWR""",6796.328,1.0,1


In [41]:
# Member-level claims aggregation
print('Aggregating claims at member level...')

member_claims = lf_claims.group_by('ADHERENT_NO').agg([
    pl.len().alias('TOTAL_CLAIM_LINES'),
    pl.col('VOU_NO').n_unique().alias('UNIQUE_CLAIMS'),
    pl.col('SUM_OF_NETBILLED').sum().alias('TOTAL_BILLED'),
    pl.col('SUM_OF_NETBILLED').mean().alias('AVG_BILLED'),
    pl.col('SUM_OF_NETBILLED').max().alias('MAX_BILLED'),
    pl.col('PROV_CODE').n_unique().alias('UNIQUE_PROVIDERS'),
    pl.col('DIAG_CODE').n_unique().alias('UNIQUE_DIAGNOSES'),
    pl.col('BEN_HEAD').n_unique().alias('UNIQUE_BENEFITS'),
]).collect()

print(f'Member claims: {member_claims.shape}')
member_claims.head()

Aggregating claims at member level...
Member claims: (2523830, 9)


ADHERENT_NO,TOTAL_CLAIM_LINES,UNIQUE_CLAIMS,TOTAL_BILLED,AVG_BILLED,MAX_BILLED,UNIQUE_PROVIDERS,UNIQUE_DIAGNOSES,UNIQUE_BENEFITS
str,u32,u32,f64,f64,f64,u32,u32,u32
"""0149867300_0100149001_202210""",14,3,2151.74652,153.69618,403.4184,1,3,2
"""0127942300_0100077114_202211""",41,5,6151.99431,150.048642,547.8858,1,4,2
"""0148808900_0100418002_202301""",228,43,193079.54435,846.840107,50881.6,5,23,5
"""0149011900_0100095001_202201""",2,1,105.62475,52.812375,61.64851,1,1,1
"""0142941600_0115992018_202307""",7,2,831.27814,118.75402,378.06846,2,2,2


In [42]:
# Merge member-level data
print('Merging member-level datasets...')

df_member_level = member_base.join(
    member_claims, on='ADHERENT_NO', how='left'
).with_columns([
    # Fill nulls for members with no claims
    pl.col('TOTAL_CLAIM_LINES').fill_null(0),
    pl.col('UNIQUE_CLAIMS').fill_null(0),
    pl.col('TOTAL_BILLED').fill_null(0),
    pl.col('AVG_BILLED').fill_null(0),
    pl.col('MAX_BILLED').fill_null(0),
    pl.col('UNIQUE_PROVIDERS').fill_null(0),
    pl.col('UNIQUE_DIAGNOSES').fill_null(0),
    pl.col('UNIQUE_BENEFITS').fill_null(0),
])

# Add derived features
df_member_level = df_member_level.with_columns([
    (pl.col('TOTAL_BILLED') > 0).cast(pl.Int8).alias('IS_UTILIZER'),
    (pl.col('TOTAL_BILLED') / pl.col('TOTAL_EARNED').replace(0, None)).alias('MEMBER_LOSS_RATIO'),
])

print(f'Member-level dataset: {df_member_level.shape}')
df_member_level.head()

Merging member-level datasets...
Member-level dataset: (4201467, 19)


ADHERENT_NO,CONTRACT_NO,PLAN_ID,GENDER,NATIONALITY,PLAN_NETWORK,TOTAL_PREMIUM,TOTAL_EARNED,MONTHS_ENROLLED,TOTAL_CLAIM_LINES,UNIQUE_CLAIMS,TOTAL_BILLED,AVG_BILLED,MAX_BILLED,UNIQUE_PROVIDERS,UNIQUE_DIAGNOSES,UNIQUE_BENEFITS,IS_UTILIZER,MEMBER_LOSS_RATIO
str,str,str,str,str,str,f64,f64,u32,u32,u32,f64,f64,f64,u32,u32,u32,i8,f64
"""0145432200_0102868081_202204""","""0145432200""","""0225834""","""M""","""Saudi Arabia""","""F. NW5""",11861.773,1.0,1,8,5,766.58582,95.823228,254.408,3,3,1,1,766.58582
"""0149108100_0101984001_202303""","""0149108100""","""0236040""","""M""","""Saudi Arabia""","""G. NW6""",2970.2134,0.22,1,0,0,0.0,0.0,0.0,0,0,0,0,0.0
"""0148025900_0102210021_202305""","""0148025900""","""0238182""","""F""","""Saudi Arabia""","""B. NW1""",17913.049,1.0,1,0,0,0.0,0.0,0.0,0,0,0,0,0.0
"""0147789600_0105730012_202202""","""0147789600""","""0224732""","""M""","""Nepal""","""NWM""",3352.734,1.0,1,0,0,0.0,0.0,0.0,0,0,0,0,0.0
"""0139154000_0100272013_202208""","""0139154000""","""0229833""","""M""","""Egypt""","""A. NWR""",6796.328,1.0,1,0,0,0.0,0.0,0.0,0,0,0,0,0.0


### 4.3 Dimension Tables (for drill-down analysis)

In [43]:
# Nationality dimension - claims by nationality per contract
print('Creating nationality dimension table...')

dim_nationality = lf_member.join(
    lf_claims.select(['ADHERENT_NO', 'SUM_OF_NETBILLED']),
    on='ADHERENT_NO',
    how='left'
).group_by(['CONTRACT_NO', 'NATIONALITY']).agg([
    pl.col('ADHERENT_NO').n_unique().alias('MEMBER_COUNT'),
    pl.col('SUM_OF_NETBILLED').sum().alias('TOTAL_BILLED'),
    pl.col('SUM_OF_NETBILLED').mean().alias('AVG_BILLED'),
    pl.col('SUM_OF_NETBILLED').count().alias('CLAIM_COUNT'),
]).collect()

print(f'Nationality dimension: {dim_nationality.shape}')
dim_nationality.head()

Creating nationality dimension table...
Nationality dimension: (281578, 6)


CONTRACT_NO,NATIONALITY,MEMBER_COUNT,TOTAL_BILLED,AVG_BILLED,CLAIM_COUNT
str,str,u32,f64,f64,u32
"""0170530236""","""France""",1,0.0,null,0
"""0170573449""","""U.S.A.""",1,0.0,null,0
"""0170506806""","""U.K.""",1,0.0,null,0
"""0170560748""","""Pakistan""",1,0.0,null,0
"""0155415800""","""Saudi Arabia""",1,0.0,null,0


In [44]:
# Provider dimension - claims by provider per contract
print('Creating provider dimension table...')

dim_provider = lf_claims.group_by(['CONT_NO', 'PROV_CODE']).agg([
    pl.col('VOU_NO').n_unique().alias('CLAIM_COUNT'),
    pl.col('SUM_OF_NETBILLED').sum().alias('TOTAL_BILLED'),
    pl.col('SUM_OF_NETBILLED').mean().alias('AVG_BILLED'),
    pl.col('ADHERENT_NO').n_unique().alias('UNIQUE_MEMBERS'),
]).rename({'CONT_NO': 'CONTRACT_NO'}).collect()

# Join with provider info
dim_provider = dim_provider.join(
    df_provider.select(['PROV_CODE', 'PROV_NAME', 'PROVIDER_NETWORK', 'PROVIDER_PRACTICE', 'PROVIDER_REGION']),
    on='PROV_CODE',
    how='left'
)

print(f'Provider dimension: {dim_provider.shape}')
dim_provider.head()

Creating provider dimension table...
Provider dimension: (1135632, 10)


CONTRACT_NO,PROV_CODE,CLAIM_COUNT,TOTAL_BILLED,AVG_BILLED,UNIQUE_MEMBERS,PROV_NAME,PROVIDER_NETWORK,PROVIDER_PRACTICE,PROVIDER_REGION
str,str,u32,f64,f64,u32,str,str,str,str
"""0112468800""","""23015""",19,23164.48442,1103.070687,7,"""Al Dawa Company for Medical Services Limited""","""F. NW5""","""Pharmacy""","""Eastern"""
"""0145740300""","""23392""",1,726.88,726.88,1,"""Maternity and Child Hospital - Aziziyah - MoH""","""I. SPA""","""Hospital""","""Western"""
"""0150475200""","""23395""",2,1537.89636,219.69948,1,"""Rabigh General Hospital - MoH""","""I. SPA""","""Hospital""","""Western"""
"""0150079400""","""20247""",2,1953.0357,177.5487,2,"""Al Zafer Hospital - Najran""","""F. NW5""","""Hospital""","""Southern"""
"""0149191600""","""926476""",2,722.01899,361.009495,1,null,null,null,null


In [45]:
# Diagnosis dimension - top diagnoses per contract
print('Creating diagnosis dimension table...')

dim_diagnosis = lf_claims.group_by(['CONT_NO', 'DIAG_CODE']).agg([
    pl.len().alias('OCCURRENCE_COUNT'),
    pl.col('SUM_OF_NETBILLED').sum().alias('TOTAL_BILLED'),
    pl.col('ADHERENT_NO').n_unique().alias('UNIQUE_MEMBERS'),
]).rename({'CONT_NO': 'CONTRACT_NO'}).collect()

print(f'Diagnosis dimension: {dim_diagnosis.shape}')
dim_diagnosis.head()

Creating diagnosis dimension table...
Diagnosis dimension: (3256468, 5)


CONTRACT_NO,DIAG_CODE,OCCURRENCE_COUNT,TOTAL_BILLED,UNIQUE_MEMBERS
str,str,u32,f64,u32
"""0147349600""","""R10.0""",7,559.47045,1
"""0146350800""","""L60.3""",1,236.236,1
"""0150494500""","""R47.8""",8,4055.67239,2
"""0139430800""","""L29.9""",6,3226.4386,1
"""0145329600""","""E73.9""",4,1936.27203,1


In [48]:
# Call category dimension - call patterns per contract
print('Creating call category dimension table...')

dim_calls = lf_calls.group_by(['CONT_NO', 'CALL_CAT']).agg([
    pl.len().alias('CALL_COUNT'),
    pl.col('MBR_NO').n_unique().alias('UNIQUE_CALLERS'),
]).rename({'CONT_NO': 'CONTRACT_NO'}).collect()

print(f'Call category dimension: {dim_calls.shape}')
dim_calls.head()

Creating call category dimension table...


Call category dimension: (191423, 4)


CONTRACT_NO,CALL_CAT,CALL_COUNT,UNIQUE_CALLERS
str,str,u32,u32
"""0148715700""","""MR""",2,1
"""0149162800""","""BS""",13,8
"""0142943900""","""CP""",4,3
"""0155363900""","""CC""",1,1
"""0156125300""","""MH""",2,1


## 5. Merge Contract-Level Dataset

In [49]:
# Merge all contract-level aggregations
print('Merging contract-level datasets...')

df_contract = agg_member_contract.join(
    agg_claims_contract, on='CONTRACT_NO', how='left'
).join(
    agg_calls_contract, on='CONTRACT_NO', how='left'
).join(
    agg_preauth_contract, on='CONTRACT_NO', how='left'
).join(
    preauth_status_rates.select(['CONTRACT_NO'] + [c for c in preauth_status_rates.columns if '_RATE' in c]),
    on='CONTRACT_NO', how='left'
)

print(f'Merged contract dataset: {df_contract.shape}')
print(f'Columns: {df_contract.columns}')

Merging contract-level datasets...
Merged contract dataset: (133704, 45)
Columns: ['CONTRACT_NO', 'TOTAL_MEMBERS', 'TOTAL_PLANS', 'TOTAL_WRITTEN_PREMIUM', 'TOTAL_EARNED', 'UNIQUE_NETWORKS', 'UNIQUE_NATIONALITIES', 'MALE_COUNT', 'FEMALE_COUNT', 'CONTRACT_START', 'CONTRACT_END', 'TOTAL_CLAIM_LINES', 'UNIQUE_CLAIMS', 'TOTAL_NET_BILLED', 'AVG_NET_BILLED', 'MAX_NET_BILLED', 'STD_NET_BILLED', 'MEMBERS_WITH_CLAIMS', 'UNIQUE_PROVIDERS', 'UNIQUE_DIAGNOSES', 'UNIQUE_BENEFIT_HEADS', 'FIRST_CLAIM_DATE', 'LAST_CLAIM_DATE', 'TOTAL_CALLS', 'UNIQUE_CALLS', 'UNIQUE_CALL_CATEGORIES', 'UNIQUE_CALLERS', 'TOTAL_PREAUTH_ITEMS', 'UNIQUE_PREAUTH_EPISODES', 'MEMBERS_WITH_PREAUTH', 'TOTAL_ESTIMATED_AMT', 'AVG_ESTIMATED_AMT', 'UNIQUE_PROVIDERS_PREAUTH', 'C_RATE', 'N_RATE', 'P_RATE', 'D_RATE', 'R_RATE', 'A_RATE', 'K_RATE', 'U_RATE', 'T_RATE', 'H_RATE', '_RATE', 'W_RATE']


In [ ]:
# Fill nulls and add derived features
print('Adding derived features...')

# Fill nulls for contracts with no activity
fill_cols = [c for c in df_contract.columns if c != 'CONTRACT_NO' and df_contract[c].dtype in [pl.Float64, pl.Int64, pl.Int32, pl.Float32]]

df_contract = df_contract.with_columns([
    pl.col(c).fill_null(0) for c in fill_cols
])

# Add IVI-relevant derived features
df_contract = df_contract.with_columns([
    # Utilization metrics
    (pl.col('MEMBERS_WITH_CLAIMS') / pl.col('TOTAL_MEMBERS')).alias('UTILIZATION_RATE'),
    (pl.col('TOTAL_CLAIM_LINES') / pl.col('TOTAL_MEMBERS')).alias('CLAIMS_PER_MEMBER'),
    (pl.col('TOTAL_NET_BILLED') / pl.col('TOTAL_MEMBERS')).alias('COST_PER_MEMBER'),
    (pl.col('TOTAL_NET_BILLED') / pl.col('MEMBERS_WITH_CLAIMS').replace(0, None)).alias('COST_PER_UTILIZER'),
    
    # Loss ratio (key sustainability metric)
    (pl.col('TOTAL_NET_BILLED') / pl.col('TOTAL_EARNED').replace(0, None)).alias('LOSS_RATIO'),
    
    # Experience metrics
    (pl.col('TOTAL_CALLS') / pl.col('TOTAL_MEMBERS')).alias('CALLS_PER_MEMBER'),
    (pl.col('UNIQUE_PREAUTH_EPISODES') / pl.col('TOTAL_MEMBERS')).alias('PREAUTH_PER_MEMBER'),
    
    # Gender ratio
    (pl.col('MALE_COUNT') / pl.col('TOTAL_MEMBERS')).alias('MALE_RATIO'),
    
    # Provider diversity
    (pl.col('UNIQUE_PROVIDERS') / pl.col('MEMBERS_WITH_CLAIMS').replace(0, None)).alias('PROVIDERS_PER_UTILIZER'),
])

# Fill any new nulls
df_contract = df_contract.fill_null(0)

print(f'Final contract dataset: {df_contract.shape}')
df_contract.head()

## 6. Outlier Analysis

In [ ]:
def analyze_outliers_polars(df: pl.DataFrame, col: str):
    """Analyze outliers using IQR method with Polars."""
    stats = df.select([
        pl.col(col).quantile(0.25).alias('Q1'),
        pl.col(col).quantile(0.75).alias('Q3'),
        pl.col(col).median().alias('median'),
        pl.col(col).mean().alias('mean'),
    ]).row(0)
    
    Q1, Q3, median, mean = stats
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outlier_count = df.filter(
        (pl.col(col) < lower) | (pl.col(col) > upper)
    ).height
    
    total = df.height
    
    print(f'\n{col} - Outlier Analysis')
    print('-' * 40)
    print(f'Q1: {Q1:,.2f}, Q3: {Q3:,.2f}, IQR: {IQR:,.2f}')
    print(f'Lower bound: {lower:,.2f}, Upper bound: {upper:,.2f}')
    print(f'Outliers: {outlier_count:,} ({outlier_count/total*100:.2f}%)')
    
    return {'Q1': Q1, 'Q3': Q3, 'IQR': IQR, 'lower': lower, 'upper': upper, 'outliers': outlier_count}


# Analyze key metrics
key_metrics = ['TOTAL_NET_BILLED', 'LOSS_RATIO', 'COST_PER_MEMBER', 'UTILIZATION_RATE']

outlier_stats = {}
for metric in key_metrics:
    outlier_stats[metric] = analyze_outliers_polars(df_contract, metric)

## 7. Visualizations

In [ ]:
# Convert to pandas for plotting (contract-level is small enough)
df_plot = df_contract.to_pandas()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 1. Contract size distribution
ax = axes[0, 0]
data = df_plot['TOTAL_MEMBERS'].clip(upper=df_plot['TOTAL_MEMBERS'].quantile(0.99))
ax.hist(data, bins=50, color='steelblue', edgecolor='white')
ax.set_xlabel('Total Members')
ax.set_ylabel('Frequency')
ax.set_title('Contract Size Distribution')

# 2. Loss ratio distribution
ax = axes[0, 1]
data = df_plot['LOSS_RATIO'].clip(upper=3)
ax.hist(data, bins=50, color='coral', edgecolor='white')
ax.axvline(1.0, color='red', linestyle='--', linewidth=2, label='Break-even')
ax.set_xlabel('Loss Ratio')
ax.set_ylabel('Frequency')
ax.set_title('Loss Ratio Distribution')
ax.legend()

# 3. Utilization rate
ax = axes[0, 2]
ax.hist(df_plot['UTILIZATION_RATE'], bins=50, color='seagreen', edgecolor='white')
ax.set_xlabel('Utilization Rate')
ax.set_ylabel('Frequency')
ax.set_title('Member Utilization Rate')

# 4. Cost per member
ax = axes[1, 0]
data = df_plot['COST_PER_MEMBER'].clip(upper=df_plot['COST_PER_MEMBER'].quantile(0.99))
ax.hist(data, bins=50, color='purple', edgecolor='white')
ax.set_xlabel('Cost per Member (SAR)')
ax.set_ylabel('Frequency')
ax.set_title('Cost per Member Distribution')

# 5. Calls per member
ax = axes[1, 1]
data = df_plot['CALLS_PER_MEMBER'].clip(upper=df_plot['CALLS_PER_MEMBER'].quantile(0.99))
ax.hist(data, bins=50, color='orange', edgecolor='white')
ax.set_xlabel('Calls per Member')
ax.set_ylabel('Frequency')
ax.set_title('Calls per Member Distribution')

# 6. Written premium
ax = axes[1, 2]
data = df_plot['TOTAL_WRITTEN_PREMIUM'].clip(upper=df_plot['TOTAL_WRITTEN_PREMIUM'].quantile(0.99))
ax.hist(data, bins=50, color='teal', edgecolor='white')
ax.set_xlabel('Total Written Premium (SAR)')
ax.set_ylabel('Frequency')
ax.set_title('Written Premium Distribution')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'contract_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation heatmap for key IVI metrics
key_metrics = [
    'TOTAL_MEMBERS', 'TOTAL_WRITTEN_PREMIUM', 'TOTAL_NET_BILLED', 'LOSS_RATIO',
    'UTILIZATION_RATE', 'COST_PER_MEMBER', 'CLAIMS_PER_MEMBER', 
    'CALLS_PER_MEMBER', 'PREAUTH_PER_MEMBER'
]

corr_matrix = df_plot[key_metrics].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, 
            fmt='.2f', square=True, linewidths=0.5,
            cbar_kws={'shrink': 0.8})
plt.title('Correlation Matrix - Key Contract Metrics', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Save All Curated Datasets

In [ ]:
# Save all curated datasets
print('='*60)
print('SAVING CURATED DATASETS')
print('='*60)

# 1. Contract-level (main IVI dataset)
df_contract.write_parquet(OUTPUT_DIR / 'contract_level.parquet')
print(f'[1] Contract-level: {df_contract.shape} -> contract_level.parquet')

# 2. Member-level (individual analysis)
df_member_level.write_parquet(OUTPUT_DIR / 'member_level.parquet')
print(f'[2] Member-level: {df_member_level.shape} -> member_level.parquet')

# 3. Dimension tables
dim_nationality.write_parquet(OUTPUT_DIR / 'dim_nationality.parquet')
print(f'[3] Nationality dim: {dim_nationality.shape} -> dim_nationality.parquet')

dim_provider.write_parquet(OUTPUT_DIR / 'dim_provider.parquet')
print(f'[4] Provider dim: {dim_provider.shape} -> dim_provider.parquet')

dim_diagnosis.write_parquet(OUTPUT_DIR / 'dim_diagnosis.parquet')
print(f'[5] Diagnosis dim: {dim_diagnosis.shape} -> dim_diagnosis.parquet')

dim_calls.write_parquet(OUTPUT_DIR / 'dim_calls.parquet')
print(f'[6] Calls dim: {dim_calls.shape} -> dim_calls.parquet')

# 4. Provider reference
df_provider.write_parquet(OUTPUT_DIR / 'ref_provider.parquet')
print(f'[7] Provider ref: {df_provider.shape} -> ref_provider.parquet')

print('\n' + '='*60)
print('ALL DATASETS SAVED SUCCESSFULLY')
print('='*60)

In [ ]:
# Summary statistics for contract-level data
print('CONTRACT-LEVEL DATASET SUMMARY')
print('='*60)
print(f'Total contracts: {df_contract.height:,}')
print(f'Total features: {df_contract.width}')
print('\nKey Statistics:')
print(df_contract.select([
    pl.col('TOTAL_MEMBERS').sum().alias('Total Members'),
    pl.col('TOTAL_WRITTEN_PREMIUM').sum().alias('Total Premium'),
    pl.col('TOTAL_NET_BILLED').sum().alias('Total Claims'),
    pl.col('LOSS_RATIO').mean().alias('Avg Loss Ratio'),
    pl.col('UTILIZATION_RATE').mean().alias('Avg Utilization'),
]))

print('\nColumn List:')
for i, col in enumerate(df_contract.columns, 1):
    print(f'  {i:2}. {col}')

In [ ]:
# List all output files with sizes
print('OUTPUT FILES')
print('='*60)
total_size = 0
for f in sorted(OUTPUT_DIR.glob('*.parquet')):
    size_mb = f.stat().st_size / 1e6
    total_size += size_mb
    print(f'  {f.name:35} {size_mb:>8.2f} MB')
print('-'*60)
print(f'  {"TOTAL":35} {total_size:>8.2f} MB')

## 9. Data Access Helper Functions

Use these functions in downstream notebooks for quick data loading.

In [ ]:
# Data access functions for downstream notebooks
# Copy these to your analysis notebooks

DATA_PATH = OUTPUT_DIR  # Update path as needed

def load_contract_data() -> pl.DataFrame:
    """Load contract-level data for IVI analysis."""
    return pl.read_parquet(DATA_PATH / 'contract_level.parquet')

def load_member_data() -> pl.DataFrame:
    """Load member-level data for individual analysis."""
    return pl.read_parquet(DATA_PATH / 'member_level.parquet')

def load_dimension(name: str) -> pl.DataFrame:
    """
    Load dimension table for drill-down analysis.
    Options: 'nationality', 'provider', 'diagnosis', 'calls'
    """
    return pl.read_parquet(DATA_PATH / f'dim_{name}.parquet')

def scan_contract_data() -> pl.LazyFrame:
    """Lazy scan for memory-efficient queries on contract data."""
    return pl.scan_parquet(DATA_PATH / 'contract_level.parquet')

def scan_member_data() -> pl.LazyFrame:
    """Lazy scan for memory-efficient queries on member data."""
    return pl.scan_parquet(DATA_PATH / 'member_level.parquet')

print('Data access functions defined.')
print('')
print('Usage examples:')
print('  df = load_contract_data()           # Full contract data')
print('  df = load_member_data()             # Full member data')
print('  df = load_dimension("nationality")  # Nationality drill-down')
print('  lf = scan_contract_data()           # Lazy for large queries')

## Summary

### Data Processing Pipeline Completed

**Input Data:**
- Members: ~4.3M rows
- Claims: ~86M rows  
- Calls: ~8.9M rows
- Preauth: ~305M rows

**Output Datasets:**

| File | Description | Use Case |
|------|-------------|----------|
| `contract_level.parquet` | Aggregated contract metrics | IVI scoring, corporate analysis |
| `member_level.parquet` | Individual member features | Health patterns, demographics |
| `dim_nationality.parquet` | Claims by nationality | Nationality-based analysis |
| `dim_provider.parquet` | Claims by provider | Provider performance |
| `dim_diagnosis.parquet` | Diagnosis patterns | Health conditions analysis |
| `dim_calls.parquet` | Call patterns | Experience/service analysis |
| `ref_provider.parquet` | Provider reference data | Lookups and joins |

**Key Derived Features:**
- `LOSS_RATIO` - Claims / Earned Premium (sustainability)
- `UTILIZATION_RATE` - Members with claims / Total members
- `COST_PER_MEMBER` - Total billed / Total members
- `CALLS_PER_MEMBER` - Service interaction intensity
- `PREAUTH_PER_MEMBER` - Pre-authorization activity

**Next Steps:**
1. IVI Model Development (use `contract_level.parquet`)
2. Member Health Analysis (use `member_level.parquet`)
3. Drill-down Visualizations (use dimension tables)